In [179]:
import meshplot as mp
import numpy as np
import open3d as o3d
import pymesh as pm

from scipy.spatial.distance import cdist

Vec2i = o3d.utility.Vector2iVector
Vec3d = o3d.utility.Vector3dVector
Vec3i = o3d.utility.Vector3iVector

In [180]:
def draw(*shapes):
    # return
    pm_shapes = [s for s in shapes if type(s) is pm.Mesh]
    o3d_shapes = [s for s in shapes if s not in pm_shapes]
    for shape in pm_shapes:
        mp.plot(shape.vertices, shape.faces)
    if o3d_shapes:
        o3d.visualization.draw_geometries(o3d_shapes)

In [181]:
mesh_path = 'data/dataset/YellowToy01/yellow_push_toy_1_70000.obj'
# mesh_path = 'data/dataset/YellowToy01/yellow_push_toy_2_70000.obj'
point_cloud_path = 'data/pointcloud_sampled/YellowToy01/inputs/non_deformed_2_correspondences_zoom_2048.ply'
# point_cloud_path = 'data/pointcloud_sampled/YellowToy01/targets/deformed_2_correspondences_zoom_2048.ply'

mesh = o3d.io.read_triangle_mesh(mesh_path)
point_cloud = o3d.io.read_point_cloud(point_cloud_path)
draw(mesh, point_cloud)

In [182]:
mesh_points = np.asarray(mesh.vertices)
cloud_points = np.asarray(point_cloud.points)

# https://stackoverflow.com/questions/8317022/get-intersecting-rows-across-two-2d-numpy-arrays

nrows, ncols = cloud_points.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [cloud_points.dtype]}

common_points, ids_in_point_cloud, ids_in_mesh = np.intersect1d(cloud_points.view(dtype), mesh_points.view(dtype), return_indices=True)

common_points = common_points.view(cloud_points.dtype).reshape(-1, ncols)
print(common_points.shape)
assert common_points.shape == np.unique(cloud_points, axis=0).shape

(2048, 3)


In [183]:
mesh_v_merge_map = np.zeros(len(mesh_points), dtype=int) + len(mesh_points)

distances = cdist(mesh_points, cloud_points)
mesh_v_merge_map = distances.argmin(axis=1)

assert len(np.unique(mesh_v_merge_map, axis=0)) == len(np.unique(cloud_points, axis=0))

In [184]:
mesh_points = cloud_points[mesh_v_merge_map]
mesh.vertices = o3d.utility.Vector3dVector(mesh_points)
draw(mesh, point_cloud)

In [185]:
pm_mesh = pm.form_mesh(
    np.asarray(mesh.vertices),
    np.asarray(mesh.triangles)
)

In [186]:
print(pm_mesh.num_vertices)
pm_mesh, _ = pm.remove_duplicated_vertices(pm_mesh)
print(pm_mesh.num_vertices)
# assert mesh.num_vertices == len(cloud_points)
mesh_to_point_cloud = cdist(pm_mesh.vertices, cloud_points).argmin(axis=1)
vertices = Vec3d(cloud_points.copy())
triangles = Vec3i(mesh_to_point_cloud[pm_mesh.faces].copy())
mesh = o3d.geometry.TriangleMesh(vertices, triangles)
draw(mesh, point_cloud)

37082
2048


In [188]:
o3d.io.write_triangle_mesh(point_cloud_path[:-3] + 'obj', mesh)

[[-0.062908 -0.103463  0.055888]
 [-0.026656 -0.061906  0.03621 ]
 [-0.058424 -0.061436  0.000938]
 ...
 [-0.060192 -0.090423  0.000519]
 [-0.038048 -0.051909  0.041447]
 [-0.086844 -0.075097  0.004415]]
[[-0.062908 -0.103463  0.055888]
 [-0.026656 -0.061906  0.03621 ]
 [-0.058424 -0.061436  0.000938]
 ...
 [-0.060192 -0.090423  0.000519]
 [-0.038048 -0.051909  0.041447]
 [-0.086844 -0.075097  0.004415]]
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[[-0.077013 -0.057812  0.009795]
 [-0.077618 -0.060341  0.007062]
 [-0.079528 -0.061161  0.008935]
 ...
 [-0.046851 -0.041734  0.064022]
 [-0.043885 -0.041631  0.058802]
 [-0.049017 -0.045758  0.055411]]
